# Fabric importance
In this notebook a train test based just on week 103 is created, and tested on week 104.

### Import of libraries and datasets

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
articles = pd.read_parquet("../generated_data/articles_with_fabric_namenum.parquet")
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,7,1,0,61,5,1017,4,8243,[1],[microfibre]
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,7,1,0,61,5,1017,4,8243,[1],[microfibre]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,3,3,26,16,1021,13,28026,"[4, 13, 3]","[cotton, knit, elastic]"
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,1,0,2,15,1005,0,28025,[8],[polyester]
105539,956217002,956217,24040,265,1,2,1010016,0,9,0,...,0,1,0,18,12,1005,0,28024,[0],[jersey]
105540,957375001,957375,24041,72,48,3,1010016,0,9,0,...,1,2,2,52,25,1019,1,28023,"[5, 6]","[plastic, hair]"


In [3]:
a2 = pd.read_csv("../data/articles.csv")

In [4]:
style_cols = ["graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]
for col in style_cols:
    print(col)
    print(a2[col].unique())
    print()

graphical_appearance_name
['Solid' 'Stripe' 'All over pattern' 'Melange' 'Transparent' 'Metallic'
 'Application/3D' 'Denim' 'Colour blocking' 'Dot' 'Other structure'
 'Contrast' 'Treatment' 'Check' 'Chambray' 'Front print'
 'Glittering/Metallic' 'Mixed solid/pattern' 'Placement print'
 'Other pattern' 'Neps' 'Embroidery' 'Lace' 'Jacquard' 'Unknown' 'Argyle'
 'Slub' 'Mesh' 'Sequin' 'Hologram']

perceived_colour_master_name
['Black' 'White' 'Beige' 'Grey' 'Blue' 'Pink' 'Lilac Purple' 'Red' 'Mole'
 'Orange' 'Metal' 'Brown' 'Turquoise' 'Yellow' 'Khaki green' 'Green'
 'undefined' 'Unknown' 'Yellowish Green' 'Bluish Green']

section_name
['Womens Everyday Basics' 'Womens Lingerie'
 'Womens Nightwear, Socks & Tigh' 'Baby Essentials & Complements'
 'Men Underwear' 'Mama' 'Womens Small accessories' 'Men H&M Sport'
 'Kids Boy' 'Divided Basics' 'Girls Underwear & Basics' 'Mens Outerwear'
 'Womens Big accessories' 'Divided Accessories'
 'Womens Swimwear, beachwear' 'Divided Selected' 'Boys Underwe

In [5]:
customers = pd.read_parquet("../data/customers.parquet")
customers

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,-1,-1,0,0,49,6305
1,11246327431398957306,-1,-1,0,0,25,33726
2,18439897732908966680,-1,-1,0,0,24,3247
3,18352672461570950206,-1,-1,0,0,54,168643
4,18162778555210377306,1,1,0,1,52,168645
...,...,...,...,...,...,...,...
1371975,7551062398649767985,-1,-1,0,0,24,50351
1371976,9305341941720086711,-1,-1,0,0,21,80169
1371977,10160427316885688932,1,1,0,1,21,106737
1371978,2551401172826382186,1,1,0,1,18,111894


In [6]:
#Limit transactions to THIS FALL:
#TRAIN = WEEK 102-103 
#TEST = WEEK 104
tr = pd.read_parquet("../generated_data/trans_art_102_to_104.parquet")
tr.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
252,2020-09-03,5316172648860712200,578472001,0.076254,2,102,578472,37340,263,38,...,0,1,0,19,40,1007,9,26053,[-1],None
253,2020-09-16,6474552940938576885,578472001,0.091508,2,104,578472,37340,263,38,...,0,1,0,19,40,1007,9,26053,[-1],None
6203,2020-09-10,5701871825794564345,613246004,0.021169,1,103,613246,5006,272,0,...,0,1,0,15,0,1009,5,3016,"[11, 12]","[denim, jeans]"
10396,2020-09-21,6949189610971841595,688873001,0.003373,1,104,688873,131,258,5,...,0,1,0,11,7,1010,6,22,[-1],None
10704,2020-09-11,2430626937109945466,673911002,0.013542,1,103,673911,11338,94,16,...,6,1,0,64,20,1020,7,7204,"[3, 16]","[elastic, suede]"


In [7]:
tr_this_fall = tr[(tr.week > 102) & (tr.week < 104)]
print(tr_this_fall.shape)

tr_this_fall_test = tr[(tr.week == 104)]
print(tr_this_fall_test.shape)

(255241, 32)
(240311, 32)


In [8]:
cus_this_fall = tr_this_fall["customer_id"].unique().tolist()
len(cus_this_fall)

72019

In [9]:
cus_this_fall_test = tr_this_fall_test["customer_id"].unique().tolist()
len(cus_this_fall_test)

68984

In [10]:
art_this_fall = tr_this_fall["article_id"].unique().tolist()
len(art_this_fall)

18611

In [11]:
art_this_fall_test = tr_this_fall_test["article_id"].unique().tolist()
len(art_this_fall_test)

17986

### Articles Matrix

In [12]:
#create the correspondence between article id and index
index_to_art_id = pd.DataFrame({'article_id': art_this_fall})
index_to_art_id_test = pd.DataFrame({'article_id': art_this_fall_test})

In [13]:
index_to_art_id.sort_values(by='article_id')
index_to_art_id['original_index'] = index_to_art_id.index
index_to_art_id.head()

,article_id,original_index
0,613246004,0
1,673911002,1
2,568601006,2
3,621381011,3
4,573937001,4


In [14]:
index_to_art_id_test.sort_values(by='article_id')
index_to_art_id_test['original_index'] = index_to_art_id_test.index
index_to_art_id_test.head()

,article_id,original_index
0,578472001,0
1,688873001,1
2,568601006,2
3,573937001,3
4,480093001,4


In [15]:
#considering just the articles in the weeks we're interested in
articles_this_fall = articles[articles['article_id'].isin(art_this_fall)]
articles_this_fall_test = articles[articles['article_id'].isin(art_this_fall_test)]

In [16]:
#creating reduced datasets for the style characteristics
articles_this_fall_fabric = articles_this_fall[['article_id', 'fabric_num']]
articles_this_fall_style = articles_this_fall[["article_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]
articles_this_fall_fabric_test = articles_this_fall_test[['article_id', 'fabric_num']]
articles_this_fall_style_test = articles_this_fall_test[["article_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]

print("training")
print(articles_this_fall_fabric.shape)
print(articles_this_fall_style.shape)
print("test")
print(articles_this_fall_fabric_test.shape)
print(articles_this_fall_style_test.shape)

training
(18611, 2)
(18611, 5)
test
(17986, 2)
(17986, 5)


In [17]:
#creating matrix for fabric

from sklearn.preprocessing import MultiLabelBinarizer

#I make sure that every value inside fabric num lists is counted as a possible value
mlb = MultiLabelBinarizer(sparse_output=True)

articles_this_fall_fabric = articles_this_fall_fabric.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(articles_this_fall_fabric.pop('fabric_num')),
                index=articles_this_fall_fabric.index,
                columns=mlb.classes_))

articles_this_fall_fabric.head()

,article_id,-1,0,1,2,3,4,5,6,7,...,83,84,90,91,94,95,97,100,103,104
1,108775044,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,111565001,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,111586001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,111593001,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,111609001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#I make sure that every value inside fabric num lists is counted as a possible value
mlb = MultiLabelBinarizer(sparse_output=True)

articles_this_fall_fabric_test = articles_this_fall_fabric_test.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(articles_this_fall_fabric_test.pop('fabric_num')),
                index=articles_this_fall_fabric_test.index,
                columns=mlb.classes_))

articles_this_fall_fabric_test.head()

,article_id,-1,0,1,2,3,4,5,6,7,...,82,83,84,90,91,92,94,97,101,103
1,108775044,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,111565001,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,111586001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,111593001,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,111609001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#remove article id
articles_this_fall_fabric = articles_this_fall_fabric.drop(['article_id'], axis=1)
#remove article id
articles_this_fall_fabric_test = articles_this_fall_fabric_test.drop(['article_id'], axis=1)

In [20]:
#creating matrix for other features
one_hot_encoded = pd.get_dummies(articles_this_fall[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,graphical_appearance_name_0,graphical_appearance_name_1,graphical_appearance_name_2,graphical_appearance_name_3,graphical_appearance_name_4,graphical_appearance_name_5,graphical_appearance_name_6,graphical_appearance_name_7,graphical_appearance_name_8,graphical_appearance_name_9,...,section_name_49,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105536,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105537,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
105538,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
one_hot_encoded_test = pd.get_dummies(articles_this_fall_test[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
one_hot_encoded_test = one_hot_encoded_test.astype(int)
one_hot_encoded_test

,graphical_appearance_name_0,graphical_appearance_name_1,graphical_appearance_name_2,graphical_appearance_name_3,graphical_appearance_name_4,graphical_appearance_name_5,graphical_appearance_name_6,graphical_appearance_name_7,graphical_appearance_name_8,graphical_appearance_name_9,...,section_name_48,section_name_49,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105536,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105537,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
105538,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
#merge the two matrices
merged_matrix_art = one_hot_encoded.join(articles_this_fall_fabric)
merged_matrix_art_test = one_hot_encoded_test.join(articles_this_fall_fabric_test)

In [23]:
merged_matrix_art = csr_matrix(merged_matrix_art.values)
print(merged_matrix_art)

  (0, 0)	1
  (0, 31)	1
  (0, 79)	1
  (0, 102)	1
  (0, 108)	1
  (1, 0)	1
  (1, 29)	1
  (1, 80)	1
  (1, 102)	1
  (1, 110)	1
  (2, 0)	1
  (2, 29)	1
  (2, 80)	1
  (2, 102)	1
  (2, 107)	1
  (3, 0)	1
  (3, 29)	1
  (3, 80)	1
  (3, 102)	1
  (3, 111)	1
  (4, 0)	1
  (4, 29)	1
  (4, 80)	1
  (4, 102)	1
  (4, 107)	1
  :	:
  (18606, 102)	1
  (18606, 111)	1
  (18606, 121)	1
  (18607, 1)	1
  (18607, 43)	1
  (18607, 61)	1
  (18607, 102)	1
  (18607, 108)	1
  (18608, 6)	1
  (18608, 29)	1
  (18608, 65)	1
  (18608, 105)	1
  (18608, 111)	1
  (18608, 112)	1
  (18608, 121)	1
  (18609, 0)	1
  (18609, 29)	1
  (18609, 64)	1
  (18609, 102)	1
  (18609, 116)	1
  (18610, 0)	1
  (18610, 29)	1
  (18610, 61)	1
  (18610, 102)	1
  (18610, 108)	1


In [24]:
merged_matrix_art_test = csr_matrix(merged_matrix_art_test.values)
print(merged_matrix_art_test)

  (0, 0)	1
  (0, 31)	1
  (0, 78)	1
  (0, 101)	1
  (0, 107)	1
  (1, 0)	1
  (1, 29)	1
  (1, 79)	1
  (1, 101)	1
  (1, 109)	1
  (2, 0)	1
  (2, 29)	1
  (2, 79)	1
  (2, 101)	1
  (2, 106)	1
  (3, 0)	1
  (3, 29)	1
  (3, 79)	1
  (3, 101)	1
  (3, 110)	1
  (4, 0)	1
  (4, 29)	1
  (4, 79)	1
  (4, 101)	1
  (4, 106)	1
  :	:
  (17981, 101)	1
  (17981, 110)	1
  (17981, 120)	1
  (17982, 1)	1
  (17982, 43)	1
  (17982, 60)	1
  (17982, 101)	1
  (17982, 107)	1
  (17983, 6)	1
  (17983, 29)	1
  (17983, 64)	1
  (17983, 104)	1
  (17983, 110)	1
  (17983, 111)	1
  (17983, 120)	1
  (17984, 0)	1
  (17984, 29)	1
  (17984, 63)	1
  (17984, 101)	1
  (17984, 115)	1
  (17985, 0)	1
  (17985, 29)	1
  (17985, 60)	1
  (17985, 101)	1
  (17985, 107)	1


In [25]:
print(merged_matrix_art.shape)
print(merged_matrix_art_test.shape)

(18611, 186)
(17986, 186)


### Customer Style Matrix

In [26]:
#saving the correspondence between customer id and index
index_to_cus_id = pd.DataFrame({'customer_id': cus_this_fall})
index_to_cus_id_test = pd.DataFrame({'customer_id': cus_this_fall_test})

In [27]:
index_to_cus_id.sort_values(by='customer_id')
index_to_cus_id['original_index'] = index_to_cus_id.index
index_to_cus_id.head()

,customer_id,original_index
0,5701871825794564345,0
1,2430626937109945466,1
2,1683368475901780587,2
3,2492065466000269975,3
4,2723802329822776681,4


In [28]:
index_to_cus_id_test.sort_values(by='customer_id')
index_to_cus_id_test['original_index'] = index_to_cus_id_test.index
index_to_cus_id_test.head()

,customer_id,original_index
0,6474552940938576885,0
1,6949189610971841595,1
2,243360189732505689,2
3,755293567580339219,3
4,2805267957821275519,4


In [29]:
#creating reduced datasets for style characteristics
customers_this_fall_fabric = tr_this_fall[['customer_id', 'fabric_num']]
customers_this_fall_style = tr_this_fall[["customer_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]
customers_this_fall_fabric_test = tr_this_fall_test[['customer_id', 'fabric_num']]
customers_this_fall_style_test = tr_this_fall_test[["customer_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]

print("training")
print(customers_this_fall_fabric.shape)
print(customers_this_fall_style.shape)
print("test")
print(customers_this_fall_fabric_test.shape)
print(customers_this_fall_style_test.shape)

training
(255241, 2)
(255241, 5)
test
(240311, 2)
(240311, 5)


In [30]:
print(index_to_cus_id.shape)
print(index_to_cus_id_test.shape)

(72019, 2)
(68984, 2)


In [31]:
#creating matrix for fabric

from sklearn.preprocessing import MultiLabelBinarizer

#I make sure that every value inside fabric num lists is counted as a possible value
mlb = MultiLabelBinarizer(sparse_output=True)

customers_this_fall_fabric = customers_this_fall_fabric.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(customers_this_fall_fabric.pop('fabric_num')),
                index=customers_this_fall_fabric.index,
                columns=mlb.classes_))

customers_this_fall_fabric.head()

,customer_id,-1,0,1,2,3,4,5,6,7,...,83,84,90,91,94,95,97,100,103,104
6203,5701871825794564345,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10704,2430626937109945466,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35509,1683368475901780587,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35510,2492065466000269975,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35511,2723802329822776681,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#I make sure that every value inside fabric num lists is counted as a possible value
mlb = MultiLabelBinarizer(sparse_output=True)

customers_this_fall_fabric_test = customers_this_fall_fabric_test.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(customers_this_fall_fabric_test.pop('fabric_num')),
                index=customers_this_fall_fabric_test.index,
                columns=mlb.classes_))

customers_this_fall_fabric_test.head()

,customer_id,-1,0,1,2,3,4,5,6,7,...,82,83,84,90,91,92,94,97,101,103
253,6474552940938576885,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10396,6949189610971841595,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35695,243360189732505689,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35696,755293567580339219,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35697,2805267957821275519,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
customers_this_fall_fabric = customers_this_fall_fabric.groupby('customer_id').sum()
#customers_this_fall_fabric_test = customers_this_fall_fabric_test.groupby('customer_id').sum()
customers_this_fall_fabric.head()

,-1,0,1,2,3,4,5,6,7,8,...,83,84,90,91,94,95,97,100,103,104
customer_id,,,,,,,,,,,,,,,,,,,,,
116809474287335,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272412481300040,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
690285180337957,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1200402310946735,0,0,0,0,1,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1219588721247131,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
customers_this_fall_fabric.sort_values(by='customer_id')
#customers_this_fall_fabric_test.sort_values(by='customer_id')

,-1,0,1,2,3,4,5,6,7,8,...,83,84,90,91,94,95,97,100,103,104
customer_id,,,,,,,,,,,,,,,,,,,,,
116809474287335,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272412481300040,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
690285180337957,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1200402310946735,0,0,0,0,1,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1219588721247131,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18445340048433064259,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18445412194736247951,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18446250046654386343,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
sparse_cus_fabric = csr_matrix(customers_this_fall_fabric.values)
#sparse_cus_fabric_test = csr_matrix(customers_this_fall_fabric_test.values)
sparse_cus_fabric

<72019x79 sparse matrix of type '<class 'numpy.intc'>'
	with 255596 stored elements in Compressed Sparse Row format>

In [42]:
#creating matrix for other stye characteristics

one_hot_encoded = pd.get_dummies(tr_this_fall[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
one_hot_encoded = one_hot_encoded.astype(int)

#one_hot_encoded_test = pd.get_dummies(tr_this_fall_test[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
#one_hot_encoded_test = one_hot_encoded_test.astype(int)
#one_hot_encoded_test

In [43]:
one_hot_encoded['customer_id'] = tr_this_fall['customer_id']
#one_hot_encoded_test['customer_id'] = tr_this_fall_test['customer_id']
one_hot_encoded.head()

,graphical_appearance_name_0,graphical_appearance_name_1,graphical_appearance_name_2,graphical_appearance_name_3,graphical_appearance_name_4,graphical_appearance_name_5,graphical_appearance_name_6,graphical_appearance_name_7,graphical_appearance_name_8,graphical_appearance_name_9,...,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4,customer_id
6203,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,5701871825794564345
10704,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2430626937109945466
35509,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1683368475901780587
35510,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2492065466000269975
35511,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2723802329822776681


In [44]:
one_hot_encoded = one_hot_encoded.groupby('customer_id').sum()
#one_hot_encoded_test = one_hot_encoded_test.groupby('customer_id').sum()
one_hot_encoded.head()

,graphical_appearance_name_0,graphical_appearance_name_1,graphical_appearance_name_2,graphical_appearance_name_3,graphical_appearance_name_4,graphical_appearance_name_5,graphical_appearance_name_6,graphical_appearance_name_7,graphical_appearance_name_8,graphical_appearance_name_9,...,section_name_49,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
customer_id,,,,,,,,,,,,,,,,,,,,,
116809474287335,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
272412481300040,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
690285180337957,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
1200402310946735,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1219588721247131,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0


In [46]:
one_hot_encoded_test.shape

(17986, 106)

In [45]:
#merge the two matrices
merged_cus = pd.merge(customers_this_fall_fabric, one_hot_encoded, on="customer_id", how="inner")
merged_cus.sort_values(by='customer_id')

#merged_cus_test = pd.merge(customers_this_fall_fabric_test, one_hot_encoded_test, on="customer_id", how="inner")
#merged_cus_test.sort_values(by='customer_id')

merged_cus


,-1,0,1,2,3,4,5,6,7,8,...,section_name_49,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
customer_id,,,,,,,,,,,,,,,,,,,,,
116809474287335,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
272412481300040,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
690285180337957,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,3,0,0,0,0
1200402310946735,0,0,0,0,1,0,0,0,0,2,...,0,0,0,0,0,2,0,0,0,0
1219588721247131,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18445340048433064259,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18445412194736247951,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
18446250046654386343,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [47]:
merged_cus = merged_cus.reset_index()
#merged_cus_test = merged_cus_test.reset_index()
merged_cus

,customer_id,-1,0,1,2,3,4,5,6,7,...,section_name_49,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
0,116809474287335,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,272412481300040,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,3,0,0,0,0
2,690285180337957,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
3,1200402310946735,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,1219588721247131,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72014,18445340048433064259,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
72015,18445412194736247951,0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,3,0,0,0,0
72016,18446250046654386343,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
72017,18446420423308293068,3,1,1,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0


In [48]:
merged_cus = merged_cus.drop("customer_id", axis=1)
#merged_cus_test = merged_cus_test.drop("customer_id", axis=1)

In [ ]:
merged_cus_test.shape

In [ ]:
merged_cus_test.head()

In [49]:
merged_cus.columns = merged_cus.columns.astype(str)
#merged_cus_test.columns = merged_cus_test.columns.astype(str)

In [50]:
#scale the values between 0 and 1

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
merged_cus = scaler.fit_transform(merged_cus)
#merged_cus_test = scaler.fit_transform(merged_cus_test)

C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [51]:
matrice_sparsa_2 = csr_matrix(merged_cus)
#matrice_sparsa_2_test = csr_matrix(merged_cus_test)
print(matrice_sparsa_2)

  (0, 5)	0.017241379310344827
  (0, 14)	0.03333333333333333
  (0, 79)	0.017241379310344827
  (0, 109)	0.04
  (0, 128)	0.043478260869565216
  (0, 181)	0.017241379310344827
  (1, 0)	0.08
  (1, 5)	0.017241379310344827
  (1, 12)	0.038461538461538464
  (1, 13)	0.041666666666666664
  (1, 80)	0.1111111111111111
  (1, 83)	0.038461538461538464
  (1, 108)	0.02
  (1, 112)	0.043478260869565216
  (1, 123)	0.1111111111111111
  (1, 128)	0.13043478260869565
  (1, 181)	0.05172413793103448
  (2, 9)	0.125
  (2, 14)	0.1
  (2, 25)	0.1
  (2, 79)	0.05172413793103448
  (2, 114)	0.09090909090909091
  (2, 122)	0.18181818181818182
  (2, 128)	0.13043478260869565
  (2, 181)	0.05172413793103448
  :	:
  (72016, 110)	0.017241379310344827
  (72016, 138)	0.07142857142857142
  (72016, 181)	0.017241379310344827
  (72017, 0)	0.12
  (72017, 1)	0.03225806451612903
  (72017, 2)	0.07142857142857142
  (72017, 14)	0.03333333333333333
  (72017, 79)	0.10344827586206896
  (72017, 108)	0.06
  (72017, 111)	0.0625
  (72017, 112)	0.04

### Compute similarity between articles and customer style

In [52]:
similarity_matrix_sparse = cosine_similarity(matrice_sparsa_2,merged_matrix_art, dense_output=False)
#similarity_matrix_sparse_test = cosine_similarity(matrice_sparsa_2_test, merged_matrix_art_test, dense_output=False)

In [53]:
import util
%run util.py

In [54]:
topk = get_top_K_values(similarity_matrix_sparse, 2)
print(topk)

  (0, 6552)	0.4424876141748583
  (0, 4817)	0.4230930081723079
  (1, 18293)	0.5355863763550818
  (1, 409)	0.5120397542746213
  (2, 18173)	0.3503177966212325
  (2, 9207)	0.41816415470104085
  (3, 16662)	0.3769351336334238
  (3, 10426)	0.4071364768663226
  (4, 1749)	0.3787534002998559
  (4, 0)	0.3787534002998559
  (5, 9997)	0.4254926491889899
  (5, 7316)	0.4254926491889899
  (6, 4818)	0.6031951766103331
  (6, 4817)	0.6031951766103331
  (7, 4012)	0.31400172579548646
  (7, 2221)	0.3111828673643744
  (8, 16942)	0.4066746765153426
  (8, 6893)	0.4227667497365949
  (9, 18464)	0.4333428913338836
  (9, 11348)	0.4333428913338836
  (10, 15535)	0.4303306180986894
  (10, 15106)	0.4303306180986894
  (11, 2384)	0.4552997855378643
  (11, 1531)	0.4552997855378643
  (12, 11011)	0.5451866702501238
  :	:
  (72006, 946)	0.529366931449037
  (72007, 11032)	0.4086183157847999
  (72007, 7254)	0.39990830753100554
  (72008, 13248)	0.5568513852229303
  (72008, 13247)	0.5568513852229303
  (72009, 17125)	0.2365455713

In [ ]:
topk_test = get_top_K_values(similarity_matrix_sparse_test, 2)
print(topk_test)

In [55]:
toprank = get_top_K_ranks(similarity_matrix_sparse, 3)
print(toprank)

  (0, 4817)	2
  (0, 4818)	3
  (0, 6552)	1
  (1, 47)	3
  (1, 409)	2
  (1, 18293)	1
  (2, 9207)	1
  (2, 17146)	3
  (2, 18173)	2
  (3, 448)	3
  (3, 10426)	1
  (3, 16662)	2
  (4, 0)	1
  (4, 1748)	3
  (4, 1749)	2
  (5, 1800)	3
  (5, 7316)	1
  (5, 9997)	2
  (6, 4817)	2
  (6, 4818)	1
  (6, 7347)	3
  (7, 2221)	2
  (7, 4012)	1
  (7, 16101)	3
  (8, 6893)	1
  :	:
  (72010, 450)	3
  (72011, 6893)	2
  (72011, 16941)	1
  (72011, 17192)	3
  (72012, 5984)	1
  (72012, 9449)	2
  (72012, 10740)	3
  (72013, 9055)	1
  (72013, 17856)	3
  (72013, 18511)	2
  (72014, 10738)	1
  (72014, 10739)	2
  (72014, 10740)	3
  (72015, 1803)	3
  (72015, 4728)	1
  (72015, 13961)	2
  (72016, 422)	2
  (72016, 423)	1
  (72016, 9318)	3
  (72017, 4930)	2
  (72017, 5645)	1
  (72017, 7278)	3
  (72018, 7254)	3
  (72018, 7255)	2
  (72018, 9074)	1


In [146]:
toprank_test = get_top_K_ranks(similarity_matrix_sparse_test, 3)
print(toprank_test)

  (0, 5450)	3
  (0, 6756)	2
  (0, 6757)	1
  (1, 3235)	1
  (1, 10885)	3
  (1, 14250)	2
  (2, 2294)	2
  (2, 10895)	1
  (2, 15274)	3
  (3, 1641)	1
  (3, 2855)	3
  (3, 3034)	2
  (4, 4700)	1
  (4, 5511)	3
  (4, 7432)	2
  (5, 3305)	2
  (5, 3308)	3
  (5, 7742)	1
  (6, 207)	2
  (6, 287)	3
  (6, 14275)	1
  (7, 5085)	1
  (7, 9624)	2
  (7, 17042)	3
  (8, 10491)	2
  :	:
  (68975, 8968)	2
  (68976, 6696)	1
  (68976, 7899)	2
  (68976, 14944)	3
  (68977, 2523)	1
  (68977, 6593)	2
  (68977, 6594)	3
  (68978, 6867)	3
  (68978, 13878)	1
  (68978, 16262)	2
  (68979, 6574)	3
  (68979, 6577)	2
  (68979, 11014)	1
  (68980, 1516)	1
  (68980, 10546)	3
  (68980, 13297)	2
  (68981, 7432)	2
  (68981, 16918)	1
  (68981, 17041)	3
  (68982, 5980)	2
  (68982, 5981)	1
  (68982, 17085)	3
  (68983, 2300)	2
  (68983, 2312)	1
  (68983, 5751)	3


In [57]:
V = toprank.data 
U = toprank.indices  # rows indexes
I = toprank.indptr  # cols indexes

print("Values (V):", V[:3])
print("Rows (U):", U[:3])
print("Columns (I):", I[:3])

Values (V): [2 3 1]
Rows (U): [4817 4818 6552]
Columns (I): [0 3 6]


In [58]:
## TRAIN

num_elements = len(V)

neg_train = pd.DataFrame(columns=['id', 'index_1', 'index_2', 'index_3'])

# Per ogni primo elemento, estrai gli indici corrispondenti
for i in range(len(I) - 1):
    start_idx = I[i]
    end_idx = I[i + 1]
    
    # Gli indici corrispondenti al primo elemento
    first_element_indices = U[start_idx:end_idx]
    
    df_to_append = pd.DataFrame({'id': [i], 'index_1': [first_element_indices[0]],
                                 'index_2': [first_element_indices[1]],
                                 'index_3': [first_element_indices[2]]})
    neg_train = pd.concat([neg_train, df_to_append], ignore_index=True)

In [ ]:
Vt = toprank_test.data 
Ut = toprank_test.indices  
It = toprank_test.indptr   

print("Values (V):", Vt[:3])
print("Rows (U):", Ut[:3])
print("Columns (I):", It[:3])

In [148]:
# TEST

num_elements = len(Vt)

neg_train_test = pd.DataFrame(columns=['id', 'index_1', 'index_2', 'index_3'])

# Per ogni primo elemento, estrai gli indici corrispondenti
for i in range(len(It) - 1):
    start_idx = It[i]
    end_idx = It[i + 1]
    
    # Gli indici corrispondenti al primo elemento
    first_element_indices = Ut[start_idx:end_idx]
    
    df_to_append_test = pd.DataFrame({'id': [i], 'index_1': [first_element_indices[0]],
                                 'index_2': [first_element_indices[1]],
                                 'index_3': [first_element_indices[2]]})
    neg_train_test = pd.concat([neg_train_test, df_to_append_test], ignore_index=True)

In [59]:
neg_train.to_parquet("../generated_data/train_test/train_103.parquet", index=False) #add index=False
#neg_train_test.to_csv("../generated_data/train_test/cand_104_test") #add index=False

In [33]:
neg_train = pd.read_parquet("../generated_data/train_test/train_103.parquet")
neg_train_test = pd.read_parquet("../generated_data/train_test/cand_104_test.parquet")

In [34]:
neg_train["customer_id"] = index_to_cus_id["customer_id"]
neg_train.head()

,id,index_1,index_2,index_3,customer_id
0,0,4817,4818,6552,5701871825794564345
1,1,47,409,18293,2430626937109945466
2,2,9207,17146,18173,1683368475901780587
3,3,448,10426,16662,2492065466000269975
4,4,0,1748,1749,2723802329822776681


In [35]:
neg_train_test["customer_id"] = index_to_cus_id_test["customer_id"]
neg_train_test.head()

,Unnamed: 0,id,index_1,index_2,index_3,customer_id
0,0,0,5450,6756,6757,6474552940938576885
1,1,1,3235,10885,14250,6949189610971841595
2,2,2,2294,10895,15274,243360189732505689
3,3,3,1641,2855,3034,755293567580339219
4,4,4,4700,5511,7432,2805267957821275519


In [36]:
index_to_art_id

,article_id,original_index
0,613246004,0
1,673911002,1
2,568601006,2
3,621381011,3
4,573937001,4
...,...,...
18606,868638003,18606
18607,912028002,18607
18608,927936001,18608
18609,866945003,18609


In [37]:
index_to_art_id_test

,article_id,original_index
0,578472001,0
1,688873001,1
2,568601006,2
3,573937001,3
4,480093001,4
...,...,...
17981,533261032,17981
17982,875646001,17982
17983,757903026,17983
17984,892893001,17984


In [38]:
# find the correspondent article for indexes

neg_train['index_1_id'] = neg_train['index_1'].apply(lambda x: index_to_art_id.loc[x, 'article_id'] if x in index_to_art_id.index else None)
neg_train_test['index_1_id'] = neg_train_test['index_1'].apply(lambda x: index_to_art_id_test.loc[x, 'article_id'] if x in index_to_art_id_test.index else None)

neg_train['index_2_id'] = neg_train['index_2'].apply(lambda x: index_to_art_id.loc[x, 'article_id'] if x in index_to_art_id.index else None)
neg_train['index_3_id'] = neg_train['index_3'].apply(lambda x: index_to_art_id.loc[x, 'article_id'] if x in index_to_art_id.index else None)

neg_train_test['index_2_id'] = neg_train_test['index_2'].apply(lambda x: index_to_art_id_test.loc[x, 'article_id'] if x in index_to_art_id_test.index else None)
neg_train_test['index_3_id'] = neg_train_test['index_3'].apply(lambda x: index_to_art_id_test.loc[x, 'article_id'] if x in index_to_art_id_test.index else None)


In [39]:
neg_train.head()

,id,index_1,index_2,index_3,customer_id,index_1_id,index_2_id,index_3_id
0,0,4817,4818,6552,5701871825794564345,826498003,852748005,864055001
1,1,47,409,18293,2430626937109945466,297078001,628327005,905416001
2,2,9207,17146,18173,1683368475901780587,841434007,947168001,917765001
3,3,448,10426,16662,2492065466000269975,651591001,769748007,810170022
4,4,0,1748,1749,2723802329822776681,613246004,742947001,748853001


In [40]:
neg_train_test.head()

,Unnamed: 0,id,index_1,index_2,index_3,customer_id,index_1_id,index_2_id,index_3_id
0,0,0,5450,6756,6757,6474552940938576885,817820002,864238001,637411014
1,1,1,3235,10885,14250,6949189610971841595,787052003,781135005,937249001
2,2,2,2294,10895,15274,243360189732505689,796610003,891375001,936012003
3,3,3,1641,2855,3034,755293567580339219,689014007,743840001,763469001
4,4,4,4700,5511,7432,2805267957821275519,793185016,766402017,785913003


In [41]:
neg_train_103 = neg_train[["customer_id", 'index_1_id', 'index_2_id', 'index_3_id']]
neg_test_104 = neg_train_test[["customer_id", 'index_1_id', 'index_2_id', 'index_3_id']]
neg_test_104.head()

,customer_id,index_1_id,index_2_id,index_3_id
0,6474552940938576885,817820002,864238001,637411014
1,6949189610971841595,787052003,781135005,937249001
2,243360189732505689,796610003,891375001,936012003
3,755293567580339219,689014007,743840001,763469001
4,2805267957821275519,793185016,766402017,785913003


In [42]:
melted_df = pd.melt(neg_train_103, id_vars=['customer_id'], var_name='index_type', value_name='article_id')
melted_df.sort_values(by=['customer_id', 'article_id'], inplace=True)

melted_df_test = pd.melt(neg_test_104, id_vars=['customer_id'], var_name='index_type', value_name='article_id')
melted_df_test.sort_values(by=['customer_id', 'article_id'], inplace=True)

melted_df_test.head()

,customer_id,index_type,article_id
4062,1402273113592184,index_1_id,699923097
73046,1402273113592184,index_2_id,852773002
142030,1402273113592184,index_3_id,869585003
122655,1827730561464445,index_2_id,837106002
191639,1827730561464445,index_3_id,851606001


In [43]:
melted_df = melted_df.drop("index_type", axis=1)
melted_df_test = melted_df_test.drop("index_type", axis=1)
melted_df.head()

,customer_id,article_id
133783,116809474287335,688704004
205802,116809474287335,872982008
61764,116809474287335,933411001
206370,272412481300040,631536021
134351,272412481300040,682289009


In [44]:
#datarfame with negative samples
melted_df["purchased"] = 0
melted_df_test["purchased"] = 0

In [45]:
#dataframe with positive samples
tr_this_fall["purchased"] = 1
tr_this_fall_test["purchased"] = 1

C:\Users\lucre\AppData\Local\Temp\ipykernel_17416\712675004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_this_fall["purchased"] = 1
C:\Users\lucre\AppData\Local\Temp\ipykernel_17416\712675004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_this_fall_test["purchased"] = 1


In [46]:
tr_this_fall.week.unique()

array([103], dtype=int8)

In [47]:
tr_this_fall_test

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name,purchased
253,2020-09-16,6474552940938576885,578472001,0.091508,2,104,578472,37340,263,38,...,1,0,19,40,1007,9,26053,[-1],None,1
10396,2020-09-21,6949189610971841595,688873001,0.003373,1,104,688873,131,258,5,...,1,0,11,7,1010,6,22,[-1],None,1
35695,2020-09-16,243360189732505689,568601006,0.050831,2,104,568601,743,264,25,...,1,0,11,7,1008,20,188,[-1],None,1
35696,2020-09-16,755293567580339219,568601006,0.050831,1,104,568601,743,264,25,...,1,0,11,7,1008,20,188,[-1],None,1
35697,2020-09-16,2805267957821275519,568601006,0.050831,2,104,568601,743,264,25,...,1,0,11,7,1008,20,188,[-1],None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31788319,2020-09-22,17026193828065643786,533261032,0.033881,2,104,533261,280,256,29,...,4,1,44,2,1002,2,185,"[4, 0]","[cotton, jersey]",1
31788320,2020-09-22,17104273831947945561,875646001,0.033881,2,104,875646,3070,272,0,...,4,1,46,8,1009,5,31807,"[4, 3, 22]","[cotton, elastic, twill]",1
31788321,2020-09-22,17843582181630128591,757903026,0.015237,1,104,757903,357,80,28,...,1,0,65,29,1019,1,166,[-1],None,1
31788322,2020-09-22,17970366705591601158,892893001,0.135576,2,104,892893,29006,252,2,...,1,0,14,36,1003,3,27251,"[13, 45, 24]","[knit, cashmere, wool]",1


In [48]:
articles_this_fall

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
6,111565001,111565,19688,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,21534,[2],[nylon]
8,111586001,111586,36001,273,15,1,1010016,0,9,0,...,7,1,0,62,31,1021,13,28038,[-1],None
9,111593001,111593,36000,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,28003,[3],[elastic]
10,111609001,111609,35999,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,28005,[-1],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105534,952267001,952267,24061,304,37,7,1010013,19,9,0,...,7,1,0,62,31,1021,13,28029,"[13, 3]","[knit, elastic]"
105536,952938001,952938,24060,254,4,0,1010001,1,13,12,...,0,1,0,18,12,1005,0,28027,[0],[jersey]
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,3,3,26,16,1021,13,28026,"[4, 13, 3]","[cotton, knit, elastic]"
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,1,0,2,15,1005,0,28025,[8],[polyester]


In [49]:
articles_this_fall_test

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
6,111565001,111565,19688,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,21534,[2],[nylon]
8,111586001,111586,36001,273,15,1,1010016,0,9,0,...,7,1,0,62,31,1021,13,28038,[-1],None
9,111593001,111593,36000,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,28003,[3],[elastic]
10,111609001,111609,35999,304,37,7,1010016,0,9,0,...,7,1,0,62,31,1021,13,28005,[-1],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105534,952267001,952267,24061,304,37,7,1010013,19,9,0,...,7,1,0,62,31,1021,13,28029,"[13, 3]","[knit, elastic]"
105536,952938001,952938,24060,254,4,0,1010001,1,13,12,...,0,1,0,18,12,1005,0,28027,[0],[jersey]
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,3,3,26,16,1021,13,28026,"[4, 13, 3]","[cotton, knit, elastic]"
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,1,0,2,15,1005,0,28025,[8],[polyester]


In [50]:
#creating random values for negative samples' features
melted_df['week'] = 103
melted_df['sales_channel_id'] = np.random.choice([1, 2], size=len(melted_df))
melted_df

,customer_id,article_id,purchased,week,sales_channel_id
133783,116809474287335,688704004,0,103,1
205802,116809474287335,872982008,0,103,1
61764,116809474287335,933411001,0,103,1
206370,272412481300040,631536021,0,103,2
134351,272412481300040,682289009,0,103,1
...,...,...,...,...,...
74229,18446420423308293068,769102009,0,103,1
2210,18446420423308293068,783801001,0,103,2
201848,18446630855572834764,907399001,0,103,2
129829,18446630855572834764,909829002,0,103,2


In [51]:
melted_df_test['week'] = 104
melted_df_test['sales_channel_id'] = np.random.choice([1, 2], size=len(melted_df_test))
melted_df_test

,customer_id,article_id,purchased,week,sales_channel_id
4062,1402273113592184,699923097,0,104,2
73046,1402273113592184,852773002,0,104,1
142030,1402273113592184,869585003,0,104,2
122655,1827730561464445,837106002,0,104,2
191639,1827730561464445,851606001,0,104,2
...,...,...,...,...,...
162865,18445641720816255142,815736002,0,104,2
24897,18445641720816255142,860411004,0,104,1
74590,18446737527580148316,743215001,0,104,2
5606,18446737527580148316,784432001,0,104,2


In [52]:
#average price
avg_prices = tr_this_fall.groupby('article_id')['price'].mean().reset_index()
avg_prices_test = tr_this_fall_test.groupby('article_id')['price'].mean().reset_index()
avg_prices

,article_id,price
0,108775044,0.008458
1,111565001,0.006977
2,111586001,0.012098
3,111593001,0.011912
4,111609001,0.008569
...,...,...
18606,952267001,0.014631
18607,952938001,0.045746
18608,953450001,0.016769
18609,953763001,0.021932


In [53]:
melted_df = pd.merge(melted_df, avg_prices, on='article_id', how='left')
melted_df_test = pd.merge(melted_df_test, avg_prices_test, on='article_id', how='left')
melted_df

,customer_id,article_id,purchased,week,sales_channel_id,price
0,116809474287335,688704004,0,103,1,0.036186
1,116809474287335,872982008,0,103,1,0.025407
2,116809474287335,933411001,0,103,1,0.020426
3,272412481300040,631536021,0,103,2,0.016579
4,272412481300040,682289009,0,103,1,0.033881
...,...,...,...,...,...,...
216052,18446420423308293068,769102009,0,103,1,0.005068
216053,18446420423308293068,783801001,0,103,2,0.040237
216054,18446630855572834764,907399001,0,103,2,0.033881
216055,18446630855572834764,909829002,0,103,2,0.013542


In [54]:
melted_df_tot = melted_df.merge(articles, on="article_id")
melted_df_tot_test = melted_df_test.merge(articles, on="article_id")
melted_df_tot

,customer_id,article_id,purchased,week,sales_channel_id,price,product_code,prod_name,product_type_no,product_type_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
0,116809474287335,688704004,0,103,1,0.036186,688704,7466,262,6,...,4,4,1,47,14,1016,11,6321,[11],[denim]
1,1144082350442564646,688704004,0,103,2,0.036186,688704,7466,262,6,...,4,4,1,47,14,1016,11,6321,[11],[denim]
2,2489327249042573731,688704004,0,103,1,0.036186,688704,7466,262,6,...,4,4,1,47,14,1016,11,6321,[11],[denim]
3,2727631108836978888,688704004,0,103,2,0.036186,688704,7466,262,6,...,4,4,1,47,14,1016,11,6321,[11],[denim]
4,3953485676435648508,688704004,0,103,1,0.036186,688704,7466,262,6,...,4,4,1,47,14,1016,11,6321,[11],[denim]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216052,18405329246453191578,664647026,0,103,1,0.050831,664647,700,272,0,...,2,3,3,20,23,1009,5,101,"[11, 12]","[denim, jeans]"
216053,18405720660127377943,784926002,0,103,1,0.021720,784926,1619,254,4,...,0,1,0,16,30,1002,2,4883,"[4, 0]","[cotton, jersey]"
216054,18415329758343459932,920012001,0,103,2,0.024881,920012,6649,252,2,...,1,2,2,53,1,1005,0,5368,"[4, 8]","[cotton, polyester]"
216055,18415718720114741821,716672019,0,103,2,0.015051,716672,2450,253,9,...,0,1,0,11,7,1010,6,1021,[8],[polyester]


In [55]:
tr_this_fall = tr_this_fall.drop("t_dat", axis=1)
tr_this_fall_test = tr_this_fall_test.drop("t_dat", axis=1)

In [56]:
print(melted_df_tot.shape)
print(melted_df_tot_test.shape)

(216057, 32)
(206952, 32)


In [57]:
print(tr_this_fall.shape)
print(tr_this_fall_test.shape)

(255241, 32)
(240311, 32)


In [58]:
data_train = pd.concat([tr_this_fall, melted_df_tot])
data_train.purchased.fillna(0, inplace=True)

In [59]:
data_train.head()

,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name,purchased
6203,5701871825794564345,613246004,0.021169,1,103,613246,5006,272,0,1,...,1,0,15,0,1009,5,3016,"[11, 12]","[denim, jeans]",1
10704,2430626937109945466,673911002,0.013542,1,103,673911,11338,94,16,5,...,1,0,64,20,1020,7,7204,"[3, 16]","[elastic, suede]",1
35509,1683368475901780587,568601006,0.045746,1,103,568601,743,264,25,0,...,1,0,11,7,1008,20,188,[-1],None,1
35510,2492065466000269975,568601006,0.050831,1,103,568601,743,264,25,0,...,1,0,11,7,1008,20,188,[-1],None,1
35511,2723802329822776681,568601006,0.050831,2,103,568601,743,264,25,0,...,1,0,11,7,1008,20,188,[-1],None,1


In [60]:
data_test = pd.concat([tr_this_fall_test, melted_df_tot_test])
data_test.purchased.fillna(0, inplace=True)

In [61]:
data_train.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)
data_test.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [62]:
print(data_train.purchased.mean())
print(data_test.purchased.mean())

0.5133790300963417
0.5080826138222205


In [63]:
data_train = pd.merge(data_train, customers, on='customer_id', how='left')
data_test = pd.merge(data_test, customers, on='customer_id', how='left')

In [64]:
y = data_train.purchased
X = data_train.drop(columns=['purchased', 'fabric_name', 'fabric_num']) 
X.shape, y.shape

((443941, 35), (443941,))

In [65]:
y_test = data_test.purchased
X_test = data_test.drop(columns=['purchased', 'fabric_name', 'fabric_num']) 
X_test.shape, y_test.shape

((420656, 35), (420656,))

### Trying some classification algorithms

In [66]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [67]:
from sklearn.linear_model import LogisticRegression  
logreg = LogisticRegression(random_state=42)
logreg.fit(X, y)
y_test_pred = logreg.predict(X_test)
print(classification_report(y_test, y_test_pred))

C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    206928
           1       0.51      1.00      0.67    213728

    accuracy                           0.51    420656
   macro avg       0.25      0.50      0.34    420656
weighted avg       0.26      0.51      0.34    420656



C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier(n_estimators=25, learning_rate=0.1, random_state=42)
ada_classifier.fit(X, y)
y_test_pred = ada_classifier.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.66      0.58      0.62    206928
           1       0.64      0.72      0.67    213728

    accuracy                           0.65    420656
   macro avg       0.65      0.65      0.65    420656
weighted avg       0.65      0.65      0.65    420656



In [69]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=30, max_depth=2, random_state=42)
rf_classifier.fit(X,y)
y_test_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.69      0.51      0.59    206928
           1       0.62      0.77      0.69    213728

    accuracy                           0.64    420656
   macro avg       0.65      0.64      0.64    420656
weighted avg       0.65      0.64      0.64    420656



In [70]:
from sklearn.ensemble import GradientBoostingClassifier
clf_model = GradientBoostingClassifier(n_estimators=40, min_samples_split=2, random_state=42)
clf_model.fit(X, y)
clf_predictions = clf_model.predict(X_test)
print(classification_report(y_test, clf_predictions))

              precision    recall  f1-score   support

           0       0.75      0.67      0.71    206928
           1       0.71      0.78      0.74    213728

    accuracy                           0.73    420656
   macro avg       0.73      0.73      0.73    420656
weighted avg       0.73      0.73      0.73    420656



In [71]:
from xgboost import XGBClassifier
bst = XGBClassifier(n_estimators=30, max_depth=3, gamma=2, learning_rate=0.1, objective='binary:logistic', random_state=42)
bst.fit(X, y)
preds = bst.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.74      0.66      0.70    206928
           1       0.70      0.78      0.74    213728

    accuracy                           0.72    420656
   macro avg       0.72      0.72      0.72    420656
weighted avg       0.72      0.72      0.72    420656



### With fabric info

In [72]:
f1 = pd.read_parquet("generated_data/articles_with_first_5_fabric.parquet")
f1 = f1[["article_id", "cotton", "elastic", "jersey", "knit", "metal"]]
f1.head()

,article_id,cotton,elastic,jersey,knit,metal
0,108775015,0,0,1,0,0
1,108775044,0,0,1,0,0
2,108775051,0,0,1,0,0
3,110065001,0,0,0,0,0
4,110065002,0,0,0,0,0


In [73]:
Xf = X.merge(f1, on="article_id")
Xf_test = X_test.merge(f1, on="article_id")

In [74]:
Xf_test

,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,Active,club_member_status,fashion_news_frequency,age,postal_code,cotton,elastic,jersey,knit,metal
0,6474552940938576885,578472001,0.091508,2,104,578472,37340,263,38,0,...,1,0,1,20,57479,0,0,0,0,0
1,6949189610971841595,688873001,0.003373,1,104,688873,131,258,5,0,...,1,0,1,24,96683,0,0,0,0,0
2,30282307336388048,688873001,0.003373,2,104,688873,131,258,5,0,...,-1,0,0,28,23534,0,0,0,0,0
3,61224237963484961,688873001,0.003373,2,104,688873,131,258,5,0,...,1,0,1,29,253694,0,0,0,0,0
4,83303103105241509,688873001,0.003373,2,104,688873,131,258,5,0,...,1,0,1,44,16361,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420651,17104273831947945561,875646001,0.033881,2,104,875646,3070,272,0,1,...,1,0,1,33,68816,1,1,0,0,0
420652,10319478034229477506,875646001,0.033881,2,104,875646,3070,272,0,1,...,1,0,1,55,132741,1,1,0,0,0
420653,17843582181630128591,757903026,0.015237,1,104,757903,357,80,28,3,...,-1,0,0,57,0,0,0,0,0,0
420654,17970366705591601158,892893001,0.135576,2,104,892893,29006,252,2,0,...,1,0,1,32,36972,0,0,0,1,0


In [76]:
from sklearn.linear_model import LogisticRegression  
logreg = LogisticRegression(random_state=42)
logreg.fit(Xf, y)
y_test_pred = logreg.predict(Xf_test)
print(classification_report(y_test, y_test_pred))

C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    206928
           1       0.51      1.00      0.67    213728

    accuracy                           0.51    420656
   macro avg       0.25      0.50      0.34    420656
weighted avg       0.26      0.51      0.34    420656



C:\Users\lucre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier(n_estimators=25, learning_rate=0.1, random_state=42)
ada_classifier.fit(Xf, y)
y_test_pred = ada_classifier.predict(Xf_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.79    206928
           1       0.78      0.84      0.81    213728

    accuracy                           0.80    420656
   macro avg       0.80      0.80      0.80    420656
weighted avg       0.80      0.80      0.80    420656



In [78]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=30, max_depth=2, random_state=42)
rf_classifier.fit(Xf,y)
y_test_pred = rf_classifier.predict(Xf_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78    206928
           1       0.78      0.84      0.81    213728

    accuracy                           0.80    420656
   macro avg       0.80      0.80      0.80    420656
weighted avg       0.80      0.80      0.80    420656



In [79]:
from sklearn.ensemble import GradientBoostingClassifier
clf_model = GradientBoostingClassifier(n_estimators=40, min_samples_split=2, random_state=42)
clf_model.fit(Xf, y)
clf_predictions = clf_model.predict(Xf_test)
print(classification_report(y_test, clf_predictions))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82    206928
           1       0.82      0.86      0.84    213728

    accuracy                           0.83    420656
   macro avg       0.83      0.83      0.83    420656
weighted avg       0.83      0.83      0.83    420656



In [80]:
from xgboost import XGBClassifier
bst = XGBClassifier(n_estimators=30, max_depth=3, gamma=2, learning_rate=0.1, objective='binary:logistic', random_state=42)
bst.fit(Xf, y)
preds = bst.predict(Xf_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.86      0.76      0.81    206928
           1       0.79      0.88      0.83    213728

    accuracy                           0.82    420656
   macro avg       0.82      0.82      0.82    420656
weighted avg       0.82      0.82      0.82    420656

